In [1]:
import pandas as pd
from FRA import *
from random import choice, randint
from scipy.optimize import minimize, Bounds

In [2]:
focals, estrategias = create_regions_and_strategies(8)
data = pd.read_csv('../Data/humans_absent.csv')
data['Region'] = data['Region'].apply(lambda x: [int(y) for y in x[1:-1].split(',')])
data['RegionGo'] = data['RegionGo'].apply(lambda x: [int(y) for y in x[1:-1].split(',')])
data['Overlap'] = data['Overlap'].apply(lambda x: [int(y) for y in x[1:-1].split(' ')])
data['Sims1'] = data['Sims1'].apply(lambda x: [float(y) for y in x[1:-1].split(',')])
data.head()

,Absent1,Is_there,Dyad,Player,Round,Score,Region,RegionGo,Overlap,Sims1
0,Unicorn_Absent,Unicorn_Absent,140-615,PL1,1,23,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.671875, 0.0, 0.6666666666666666, 0.20967741..."
1,Unicorn_Absent,Unicorn_Absent,140-615,PL2,1,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.171875, 0.0, 0.16216216216216217, 0.1315789..."
2,Unicorn_Absent,Unicorn_Absent,140-615,PL1,9,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,Unicorn_Absent,Unicorn_Absent,140-615,PL2,9,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.421875, 0.0, 0.3409090909090909, 0.25531914..."
4,Unicorn_Absent,Unicorn_Absent,140-615,PL1,10,32,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [3]:
mean_sims = []
for i in range(8):
    data['S_' + nameRegion(i + 1)] = data['Sims1'].apply(lambda x: x[i])
    media = data['S_' + nameRegion(i + 1)].mean()
    print("Mean sim to " + nameRegion(i + 1), media)
    mean_sims.append(media)

mean_sims

Mean sim to ALL 0.4912203577170418
Mean sim to NOTHING 0.07877813504823152
Mean sim to BOTTOM 0.33528244810093094
Mean sim to TOP 0.32162847834366115
Mean sim to LEFT 0.33059026666494695
Mean sim to RIGHT 0.32773535446135266
Mean sim to IN 0.3347388392498303
Mean sim to OUT 0.2758428071738139


[0.4912203577170418,
 0.07877813504823152,
 0.33528244810093094,
 0.32162847834366115,
 0.33059026666494695,
 0.32773535446135266,
 0.3347388392498303,
 0.2758428071738139]

In [3]:
def get_strategy(sims):
    sims = [float(x) for x in sims]
    a = sims[0]
    b = sims[2]
    t = sims[3]
    l = sims[4]
    r = sims[5]
    A = int(np.ceil(64 * a))
    B = int(b * (A + 32) / (1 + b))
    T = int(t * (A + 32) / (1 + t))
    L = int(l * (A + 32) / (1 + l))
    R = int(r * (A + 32) / (1 + r))
#    print(a, b, t, l, r)
#    print("A = ", A)
#    print("A int B =", B)
#    print("A int T =", T)
#    print("A int L =", L)
#    print("A int R =", int(r * (A + 32) / (1 + r)))
    Ma = min(T, L, 16, (16 + T + L - A))
    Mi = max(0, T - R, L - 16, T - 16)
#    print("Min(T, L, 16, (16 + T + L - A)) =", Ma)
#    print("Max(0, T - R, L - 16, T - 16) =", Mi)
    x = randint(Mi, Ma)
    y = T - x
    u = L - x
    v = A - x - y - u
#    print("x =", x)
#    print("y =", y)
#    print("u =", u)
#    print("v =", v)
    X = list(range(0,4)) + list(range(8,12)) + list(range(16,20)) + list(range(24, 28))
    Y = list(range(4,8)) + list(range(12,16)) + list(range(20,24)) + list(range(28, 32))
    U = list(range(32,36)) + list(range(40,44)) + list(range(48,52)) + list(range(56, 60))
    V = list(range(36,40)) + list(range(44,48)) + list(range(52,56)) + list(range(60, 64))
    X = list(np.random.choice(X, x, replace=False)) if x > 0 else []
    Y = list(np.random.choice(Y, y, replace=False)) if y > 0 else []
    U = list(np.random.choice(U, u, replace=False)) if u > 0 else []
    V = list(np.random.choice(V, v, replace=False)) if v > 0 else []    
    region = X + Y + U + V
#    print(region)
    return region

def shaky_hand(strategy, p=2):
    outs = np.random.choice(strategy, p) if len(strategy) > 0 else []
    complement = [i for i in range(64) if i not in strategy]
    ins = np.random.choice(complement, p) if len(complement) > 0 else []
    strategy = [i for i in strategy if i not in outs] + list(ins)
    return [i for i in strategy]

def err2_regions(region1, region2):
    sims1 = [sim_consist(region1, k) for k in focals]     
    sims2 = [sim_consist(region2, k) for k in focals]
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_sim_region(sims1, region):
    sims2 = [sim_consist(region, k) for k in focals]
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_sims(sims1, sims2):
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_model(df, pl, modelParameters, Num_Loc, focals, estrategias, p):
    df['SimsPred'] = df.apply(lambda x: attractiveness(
                            x['Region'], x['Score'], x['Overlap'], 
                            pl, modelParameters, Num_Loc, focals), 
                            axis = 1)
    df['SimsPred'] = df['SimsPred'].apply(lambda x: shaky_hand(estrategias[np.argmax(x) + 1], p))    
    df['SimsPred'] = df['SimsPred'].apply(lambda x: code2Vector(x, Num_Loc))
    df['SimsPred'] = df.apply(lambda x: err2_sim_region(x['Sims1'], x['SimsPred']), axis=1)
    return df['SimsPred'].sum()

def MSD(params):
    
    global data
    global focals
    global estrategias
    
    pl = 0
    Num_Loc = 8
    p = 4

#    er = err2_model(data, pl, modelParameters, Num_Loc, focals, estrategias, p)
#    print("Parametros: " + str(params) + "\nError: " + str(er))    
#    return er

    return err2_model(data, pl, modelParameters, Num_Loc, focals, estrategias, p)

def optimizar(parametros, minimos, maximos):
    x0 = np.array(parametros)
    bounds = Bounds(minimos, maximos)
    print("Finding fitting parameters. Please wait...")
    res = minimize(MSD, x0, method='trust-constr', bounds=bounds, options={'verbose':1})
    return res.x

In [53]:
mean_strategy = get_strategy(mean_sims)
mean_region = code2Vector(mean_strategy, 8)
imprime_region(mean_region)

[0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 0, 1, 0]
[0, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 0, 1, 1]


In [54]:
strategy = shaky_hand(mean_strategy, 0)
region1 = code2Vector(strategy, 8)
imprime_region(region1)

[0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 0, 1, 0]
[0, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 0, 1, 1]


In [55]:
err2(region, region1)

0.014207278720910514

In [66]:
data['SimsP'] = data['Sims1'].apply(lambda x: [sim_consist(mean_region, k) for k in focals])
Baseline = data.apply(lambda x: err2_sims(x['Sims1'], x['SimsP']), axis=1).sum()
Baseline

622.531531233805

In [109]:
pl = 0
modelParameters = [0, 0, 0, 0, # Biases
                   1, 100, 31.5, # Win Stay
                   1, 100, 0.9, # Attraction
                   1, 100, 0.9 # Repulsion to complement
                  ]
Num_Loc = 8

n = 10
region = data.iloc[n]['Region']
overlap = data.iloc[n]['Overlap']
score = data.iloc[n]['Score']
imprime_region(region)
print("Score", score)
attrs = attractiveness(region, score, overlap, pl, modelParameters, Num_Loc, focals)
print(attrs)
reg_selected = np.argmax(attrs)
print(reg_selected)
strategy = shaky_hand(estrategias[reg_selected + 1], 0)
region1 = code2Vector(strategy, 8)
imprime_region(region1)
regionGo = data.iloc[n]['RegionGo']
imprime_region(regionGo)
print(err2_regions(region1, regionGo))

[1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
Score 14
[4.37500000e-01 1.34285442e-27 3.04347826e-01 3.04347826e-01
 3.04347826e-01 3.04347826e-01 6.79872709e-12 1.00000000e+00]
7
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
0.0


In [110]:
for p in range(6):
    print("Shaky hand value:", p, "     Error:", err2_model(data, pl, modelParameters, Num_Loc, focals, estrategias, p))

Shaky hand value: 0      Error: 566.2331133047865
Shaky hand value: 1      Error: 592.5189382644508
Shaky hand value: 2      Error: 572.4546488481399
Shaky hand value: 3      Error: 556.5313895995391
Shaky hand value: 4      Error: 544.1062991414087
Shaky hand value: 5      Error: 535.5173776340328


In [107]:
modelParameters = [0, 0, 0, 0, # Biases
                   1, 100, 31.5, # Win Stay
                   1, 20, 0.9, # Attraction
                   1, 20, 0.7 # Repulsion to complement
                  ]
err2_model(data, pl, modelParameters, Num_Loc, focals, estrategias, 4)

561.9033779013166

In [5]:
modelParameters = [0.01, 0.01, 0.01, 0.01, # Biases
                   0, 0, 0, # Win Stay
                   0, 0, 0, # Attraction
                   0, 0, 0 # Repulsion to complement
#                   1, 100, 31.5, # Win Stay
#                   1, 20, 0.9, # Attraction
#                   1, 20, 0.7 # Repulsion to complement
                  ]
minimos = [0] * 13
maximos = [0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#maximos = [0.25, 0.25, 0.25, 0.25, 1, 100, 32, 1, 100, 1, 1, 100, 1]
optimizar(modelParameters, minimos, maximos)

KeyboardInterrupt: 